# Implementing PWM with GPIO using MicroBlaze

In [1]:
import time
import asyncio
from datetime import datetime

In [2]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

## Defining functions to write, read, & reset GPIO pins

In [3]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

gpio bgr[3];

// initialize and configure GPIO pins for output
unsigned int init_gpio() {
    bgr[0] = gpio_open(1);
    gpio_set_direction(bgr[0], GPIO_OUT);
    bgr[1] = gpio_open(2);
    gpio_set_direction(bgr[1], GPIO_OUT);
    bgr[2] = gpio_open(3);
    gpio_set_direction(bgr[2], GPIO_OUT);
    return 0;
}

//Function to turn on/off a selected pin of PMODB
unsigned int write_gpio(unsigned int pin, unsigned int val) {
    if (pin < 1 || pin > 3)
        return 1;
    gpio_write(bgr[pin-1], val);
    return 0;
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Function to reset GPIO pins
unsigned int reset_gpio() {
    write_gpio(1, 0); // blue
    write_gpio(2, 0); // green
    write_gpio(3, 0); // red
    return 0;
}

In [4]:
init_gpio()

0

In [5]:
# color-pin mappings for the pinmask
BLUE = 0b001
GREEN = 0b010
RED = 0b100

async def pwm(freq, duty_cycle, pinmask, duration=0.25):
    """
    Implements PWM for the PMOD on the PYNQ board
    
    :param freq: The frequency of one PWM cycle
    :param duty_cycle: The "ON" percentage per cycle
                       of all GPIO pins
    :param pinmask: 3-bit on/off mask for BGR sub-LEDs
    :param duration: time for the pwm to run for
    """
    assert 0.0 <= duty_cycle and duty_cycle <= 1.0
    assert freq > 0
    
    period = 1.0 / freq
    
    duration_roundcount = int(duration / period)
    
    time_on = period * duty_cycle
    time_off = period * (1.0 - duty_cycle)

    try:
        for i in range(duration_roundcount):
            for i in range(1,4):
                if (pinmask >> (i-1)) & 1:
                    write_gpio(i, 1)
            await asyncio.sleep(time_on)
            reset_gpio()
            await asyncio.sleep(time_off)
    except KeyboardInterrupt as e:
        reset_gpio()
    return

In [6]:
# duty cycles for perceived brightness follow a logarithmic scale
nofourth = 0
onefourth = 0.001 # 25% brightness, 1ms
twofourth = 0.01 # 50% brightness, 10ms
threefourth = 0.1 # 75% brightness, 100ms
fourfourth = 1.0 # 100% brightness, 1000ms (1s, always on)

In [7]:
btns = base.btns_gpio

button_color_mapping = {
    1 << 0: RED,
    1 << 1: GREEN,
    1 << 2: BLUE,
}

color_str_mapping = {
    RED: 'RED',
    GREEN: 'GREEN',
    BLUE: 'BLUE',
}

button = 1 # red
color = RED

async def blink(dc, pinmask, duration=1.0):
    await pwm(100, dc, pinmask, duration)
    await asyncio.sleep(duration)

async def button_handler():
    global button
    while True:
        button = btns.read()
        await asyncio.sleep(0.01)
    
async def blink_handler():
    global button, color

    on = True
    while True:
        try:
            color = button_color_mapping[button]
            print(f'Changing to color: {color_str_mapping[color]}')
            on = True
        except KeyError:
            if button == (1 << 3):
                on = False
        except Exception as e:
            print(f'{e}')
        finally:
            if on:
                await pwm(100, twofourth, color, 1.0)
        print(f'Color: {color_str_mapping[color]}, LAst BTN Read: {button}')
        await asyncio.sleep(1.0)

def main_loop():
    # listen for button press
    # blink LED
    loop = asyncio.new_event_loop()
    loop.create_task(button_handler())
    loop.create_task(blink_handler())
    loop.run_forever()
    loop.close()

main_loop()

Color: RED, LAst BTN Read: 0
Color: RED, LAst BTN Read: 0
Color: RED, LAst BTN Read: 2
Changing to color: GREEN
Color: GREEN, LAst BTN Read: 0
Color: GREEN, LAst BTN Read: 4
Changing to color: BLUE
Color: BLUE, LAst BTN Read: 0
Color: BLUE, LAst BTN Read: 1
Changing to color: RED
Color: RED, LAst BTN Read: 0
Color: RED, LAst BTN Read: 8
Color: RED, LAst BTN Read: 8
Color: RED, LAst BTN Read: 0
Changing to color: GREEN
Color: GREEN, LAst BTN Read: 0
Color: GREEN, LAst BTN Read: 4
Changing to color: BLUE
Color: BLUE, LAst BTN Read: 0
Color: BLUE, LAst BTN Read: 8
Color: BLUE, LAst BTN Read: 8
Color: BLUE, LAst BTN Read: 0
Color: BLUE, LAst BTN Read: 0
Color: BLUE, LAst BTN Read: 0
Color: BLUE, LAst BTN Read: 0
Color: BLUE, LAst BTN Read: 0


KeyboardInterrupt: 

In [8]:
# Go from 25% brightness to 100% in 25% increments
loop1 = asyncio.new_event_loop()
loop2 = asyncio.new_event_loop()
loop3 = asyncio.new_event_loop()
loop4 = asyncio.new_event_loop()

fut1 = loop1.create_task(pwm(freq=100, duty_cycle=onefourth, pinmask=0b111))
fut2 = loop2.create_task(pwm(freq=100, duty_cycle=twofourth, pinmask=0b111))
fut3 = loop3.create_task(pwm(freq=100, duty_cycle=threefourth, pinmask=0b111))
fut4 = loop4.create_task(pwm(freq=100, duty_cycle=fourfourth, pinmask=0b111))

loop1.run_until_complete(fut1)
reset_gpio()
time.sleep(1)
loop2.run_until_complete(fut2)
reset_gpio()
time.sleep(1)
loop3.run_until_complete(fut3)
reset_gpio()
time.sleep(1)
loop4.run_until_complete(fut4)
